In [12]:
%pip uninstall chromadb --yes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: chromadb 0.4.23
Uninstalling chromadb-0.4.23:
  Successfully uninstalled chromadb-0.4.23
Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/cc/63/b7d76109331318423f9cfb89bd89c99e19f5d0b47a5105439a629224d297/chromadb-0.4.24-py3-none-any.whl.metadata
  Obtaining dependency information for opentelemetry-instrumentation==0.44b0 from https://files.pythonhosted.org/packages/bf/e3/b2a8260862d278ac2505c8cb61d11185349f5ea537946220f5c7d738de2a/opentelemetry_instrumentation-0.44b0-py3-none-any.whl.metadata
  Using cached opentelemetry_instrumentation-0.44b0-py3-none-any.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.7 MB/s eta 0:00:00a 0:00:01
Using cached opentelemetry_instrumentation-0.44b0-py3-none-any.whl (28 kB)
  Attempting uninstall: opentelemetry-instrumentation
    Found existing installation: opentelemetry-instrumentation 0.43b0
    Uninstalling opentelemetry-instrumentation-0.43b0:
      Successfully uninstalled opentelemetry-instrumentation-0.43b0
ERROR: pip's dependency resolver does not curren

In [1]:
%pip show chromadb

Name: chromadb
Version: 0.4.24
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /Users/dhruvanand/miniforge3/lib/python3.10/site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: llama-index-vector-stores-chroma
Note: you may need to restart the kernel to use updated packages.


In [1]:
import chromadb

In [ ]:
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.PersistentClient()

In [2]:
client.list_collections()
# client.delete_collection('test3')

[]

In [8]:
# Create collection. get_collection, get_or_create_collection, delete_collection also available!
collection2 = client.get_or_create_collection("test")

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection2.add(
    documents=[
        "This is document1",
        "This is document2",
    ],  # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}],  # filter on these!
    ids=["doc1", "doc2"],  # unique for each doc
    embeddings=[[1, 2, 3], [4, 5, 6]],  # optional, we can also embed for you
)

# Query/search 2 most similar results. You can also .get by id
results = collection2.query(
    query_embeddings=[[1, 2, 3]],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2


In [9]:
results

{'ids': [['doc3', 'doc1']],
 'distances': [[0.0, 0.0]],
 'metadatas': [[{'source': 'notion'}, {'source': 'notion'}]],
 'embeddings': None,
 'documents': [['This is document1', 'This is document1']],
 'uris': None,
 'data': None}

In [11]:
coll = client.get_collection("test")

In [12]:
coll.count()

4

In [15]:
client.delete_collection("test")  # delete collection

In [7]:
client.list_collections()  # list collections

[Collection(name=test)]

In [11]:
collection.model_json_schema()

{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'id': {'format': 'uuid', 'title': 'Id', 'type': 'string'},
  'metadata': {'anyOf': [{'type': 'object'}, {'type': 'null'}],
   'default': None,
   'title': 'Metadata'},
  'tenant': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Tenant'},
  'database': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Database'}},
 'required': ['name', 'id'],
 'title': 'Collection',
 'type': 'object'}

In [7]:
pclient = chromadb.PersistentClient("~/.chroma4")  # persistent client

pcol = pclient.create_collection("test3")

In [8]:
pcol.add(
    documents=["This is document1", "This is document2"],
    metadatas=[{"source": "notion"}, {"source": "google-docs"}],
    ids=["doc1", "doc2"],
    embeddings=[[1, 2, 3], [4, 5, 6]],
)

In [9]:
pcol.query(
    query_texts=["This is a query document"],
    # n_results=2,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 3

In [13]:
client.get_settings().persist_directory

'./chroma'

In [2]:
client2 = chromadb.PersistentClient(
    "/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/chroma"
)

In [3]:
client2.get_settings().persist_directory

'/Users/dhruvanand/Code/vector-io/src/vdf_io/notebooks/chroma'

In [4]:
client2.list_collections()

[Collection(name=test),
 Collection(name=vdf_2024_9-11),
 Collection(name=movie_reviews)]

In [30]:
collection2 = client2.get_or_create_collection("test")
collection2.count()

4

In [6]:
coll2 = client2.get_collection("vdf_2024_9-11")

In [7]:
coll2.count()

308000

In [9]:
coll2.query(query_embeddings=[0 for i in range(384)], n_results=2)

{'ids': [['209770', '209839']],
 'distances': [[22.970975875854492, 22.970975875854492]],
 'metadatas': [[{'ALPHA': 'ALPHA',
    'Category': '',
    'Message': 'applmgr@hp05fat.finance.sel.sony.com|   SMOESOPG in PROD on 11-SEP-2001|Orders Transmitted: ECOMM 151, INTEK 166, Orders Booked: ECOMM 131, INTEK 148, WO Opened: 39 WO Completed: 21 Pick Released: 770 Deliveries Sent to DMPLUS: 3P1 726, SDC',
    'Message Length': 240,
    'Priority': 'C',
    'Provider': 'Skytel',
    'Provider ID': '005384363'},
   {'ALPHA': 'ALPHA',
    'Category': '',
    'Message': 'applmgr@hp05fat.finance.sel.sony.com|   SMOESOPG in PROD on 11-SEP-2001|Orders Transmitted: ECOMM 151, INTEK 166, Orders Booked: ECOMM 131, INTEK 148, WO Opened: 39 WO Completed: 21 Pick Released: 770 Deliveries Sent to DMPLUS: 3P1 726, SDC',
    'Message Length': 240,
    'Priority': 'C',
    'Provider': 'Skytel',
    'Provider ID': '005384427'}]],
 'embeddings': None,
 'documents': [['applmgr@hp05fat.finance.sel.sony.com|   S

In [20]:
collection2.add(
    documents=["This is document1", "This is document2"],
    metadatas=[{"source": "notion"}, {"source": "google-docs"}],
    ids=["doc5", "doc6"],
    embeddings=[[1, 2, 3], None],
)

ValueError: Expected each embedding in the embeddings to be a list, got [[1, 2, 3], None]